## Table of Contents
1. [Load Data](#Load-Data-And-Analysis)
2. [Preprocessing](#preprocessing)
3. [Model Selection](#model-selection)
4. [Visualization](#visualization)
5. [Summary & Key Insights](#-Notebook-Summary-&-Key-Insights)

# Load Data And Analysis 

In [4]:
import kagglehub

path = kagglehub.dataset_download("gpreda/bbc-news")
print("Path to dataset files:", path)

100%|██████████| 3.64M/3.64M [00:01<00:00, 2.60MB/s]

Extracting files...
Path to dataset files: C:\Users\AIJimmy\.cache\kagglehub\datasets\gpreda\bbc-news\versions\1007


In [6]:
import pandas as pd

df = pd.read_csv(path + '/bbc_news.csv')

df.head()

,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...


In [7]:
df.drop(columns=['pubDate','link','guid'], inplace=True)
df.head()

,title,description
0,Ukraine: Angry Zelensky vows to punish Russian...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,Consumers are feeling the impact of higher ene...


# Preprocessing

In [32]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AIJimmy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AIJimmy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AIJimmy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\AIJimmy\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [29]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from num2words import num2words
from nltk.corpus import wordnet
import contractions
from nltk.corpus import stopwords
import re 

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [30]:
# Function to map NLTK POS tags to WordNet POS tags
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [35]:
def preprocess_text(text):
    text = text.lower()
    expanded_text = contractions.fix(text)
    split_dot_text = expanded_text.replace('.', ' . ')
    html_tag_filter = re.sub(r'<[^>]*>', '', split_dot_text)
    a_z_digits_string_only = re.sub(r'[^a-zA-Z0-9]+', ' ', html_tag_filter)
    converted_num = "".join(num2words(int(word)) if word.isdigit() else word for word in a_z_digits_string_only)
    word_tokens = word_tokenize(converted_num)
    remove_stop_words = [w for w in word_tokens if not w in stop_words]
    tagged = nltk.tag.pos_tag(remove_stop_words)
    
    lemmatized_words = []
    for word, tag in tagged:
        wordnet_pos = get_wordnet_pos(tag) or wordnet.NOUN
        lemmatized_word = lemmatizer.lemmatize(word, pos=wordnet_pos)
        lemmatized_words.append(lemmatized_word)
    
    return lemmatized_words


In [36]:
df["preprocessed_title"] = df["title"].apply(preprocess_text)
df["preprocessed_description"] = df["description"].apply(preprocess_text)

In [37]:
texts = df['preprocessed_title'] + df['preprocessed_description']
texts

0        [ukraine, angry, zelensky, vow, punish, russia...
1        [war, ukraine, take, cover, town, attack, jere...
2        [ukraine, war, catastrophic, global, food, one...
3        [manchester, arena, bomb, saffie, roussos, par...
4        [ukraine, conflict, oil, price, soar, high, le...
                               ...                        
42110    [highlight, wale, make, history, dublin, watch...
42111    [gang, jail, twozerozerom, cocaine, banana, bo...
42112    [scottish, budget, present, huge, challenge, s...
42113    [celebration, wale, make, history, qualifying,...
42114    [school, tell, muslim, girl, safe, three, chil...
Length: 42115, dtype: object

In [107]:
from gensim import corpora

corpus = [' '.join(text) for text in texts.tolist()]
dictionary = corpora.Dictionary(texts)
gensim_corpus = [dictionary.doc2bow(text) for text in texts]

print("dictionary",dictionary)

dictionary Dictionary<30864 unique tokens: ['angry', 'atrocity', 'civilian', 'country', 'forget']...>


# Model selection

In [113]:
from gensim.models import LdaModel , CoherenceModel

lda_model = LdaModel(corpus=gensim_corpus, id2word=dictionary, num_topics=8, random_state=42, passes=20)
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
print("Number of topics:", 10, "LDA Coherence Score:", coherence_model_lda.get_coherence())

Number of topics: 10 LDA Coherence Score: 0.3832107815003515


In [101]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(corpus)

nmf = NMF(n_components=10, random_state=42, init='nndsvd', max_iter=500)
nmf.fit(tfidf)

NMF(init='nndsvd', max_iter=500, n_components=10, random_state=42)

In [102]:
import numpy as np
W = nmf.fit_transform(tfidf)
H = nmf.components_
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
topics = [[feature_names[i] for i in topic.argsort()[:-6:-1]] for topic in H]

coherence_model = CoherenceModel(
    topics=topics,
    texts=texts,
    dictionary=dictionary,
    coherence='c_v'
)
coherence_score = coherence_model.get_coherence()
print("NMF Coherence Score:", coherence_score)

NMF Coherence Score: 0.8357806750427382


In [111]:
def show_topics(model, num_words=10):
    topics = []
    for t in range(model.num_topics):
        wp = model.show_topic(t, topn=num_words)
        topic_words = ', '.join([w for w, p in wp])
        topics.append((t, topic_words))
    return pd.DataFrame(topics, columns=['topic', 'top_words'])

show_topics(lda_model, num_words=12)


,topic,top_words
0,0,"say, gaza, bbc, israel, israeli, hamas, tell, ..."
1,1,"euro, king, paper, tax, labour, sunak, lead, s..."
2,2,"election, year, first, day, scottish, vote, ne..."
3,3,"police, die, year, say, man, death, old, kill,..."
4,4,"win, open, paris, first, world, gold, one, rec..."
5,5,"say, trump, president, ukraine, war, minister,..."
6,6,"people, find, arrest, year, home, budget, miss..."
7,7,"school, show, film, medium, video, social, cam..."
8,8,"say, uk, right, strike, government, plan, stor..."
9,9,"bbc, say, collapse, life, cancer, care, ban, h..."


# Visualization

In [121]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

pyLDAvis.enable_notebook()

try:
    corpus_for_vis = gensim_corpus 
except NameError:
    try:
        corpus_for_vis = [dictionary.doc2bow(text) for text in texts]
    except Exception as e:
        raise RuntimeError('Could not construct a gensim corpus for visualization.') from e

vis = gensimvis.prepare(lda_model, corpus_for_vis, dictionary)
vis




PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.134980 -0.022034       1        1  19.936607
4      0.158762  0.104578       2        1  17.151818
3     -0.158555 -0.000745       3        1  12.991011
7     -0.166297  0.101872       4        1  12.775722
6     -0.170520  0.025209       5        1  10.216567
2      0.192651  0.052563       6        1   9.472764
0      0.079287 -0.362361       7        1   9.282229
1      0.199651  0.100918       8        1   8.173282, topic_info=        Term         Freq        Total Category  logprob  loglift
437      win  3311.000000  3311.000000  Default  30.0000  30.0000
98        uk  3256.000000  3256.000000  Default  29.0000  29.0000
657   police  1837.000000  1837.000000  Default  28.0000  28.0000
1126   final  2063.000000  2063.000000  Default  27.0000  27.0000
675      cup  1900.000000  1900.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
866    sport   267.062078   802.424981   Topic8  -5.4761   1.4041
25      take   275.660495  2439.484791   Topic8  -5.4444   0.3239
448     week   261.398404  1389.109128   Topic8  -5.4975   0.8339
10       say   263.820001  9865.061782   Topic8  -5.4883  -1.1172
2070    tour   245.230357   587.687861   Topic8  -5.5614   1.6303

[442 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1855      3  0.997456         abuse
1042      7  0.996998         actor
4959      7  0.996259         admit
2426      6  0.996658        africa
738       3  0.102005           age
...     ...       ...           ...
55        5  0.718779         young
55        8  0.120444         young
1635      2  0.998995          zero
1759      4  0.462393  zerozerozero
1759      5  0.536509  zerozerozero

[651 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 5, 4, 8, 7, 3, 1, 2])

# Notebook Summary & Key Insights

## Summary
- Goal: discover hidden topics/themes from a collection of news articles (BBC dataset). The notebook demonstrates a practical pipeline: data load → text cleaning → tokenization & lemmatization → create dictionary & corpus → train topic models (LDA, NMF) → evaluate coherence → visualize with pyLDAvis.

## Data & Preprocessing
- Combined title and description into a single `text_combined` column and tokenized into `tokens`.
- Preprocessing steps used: lowercasing, punctuation/HTML removal, contraction expansion, number→words (where used), stopword removal, POS-aware lemmatization (WordNet), and filtering short tokens.
- Notes: check for and remove any empty documents after cleaning before building dictionary/corpus.

## Modeling Summary
- LDA (gensim) was trained on the bag-of-words corpus. Default example used num_topics = 8 and multiple passes.
- NMF (scikit-learn with TF‑IDF) was used as an alternate non-probabilistic method; topics were extracted from the NMF `components_` matrix.
- Coherence (c_v) was computed for LDA (and optionally for NMF-derived topics) to provide a quantitative quality check.

## Visualization
- Topics visualized with pyLDAvis for interactive exploration of topic-term relationships and document-topic distributions.
- A helper table/function prints top-n words per topic for quick inspection.

## Key Findings & Practical Notes
- Coherence helps select a reasonable `num_topics` — run a small grid (e.g., 4–20) and pick the number with the best coherence while preserving interpretability.
- LDA gives probabilistic topic assignments (good for soft clustering); NMF on TF‑IDF often yields more coherent sparse topics for short documents.
- If training is slow, use `LdaMulticore` (gensim) and tune `passes`, `iterations`, and `chunksize` for speed/quality tradeoffs.
- pyLDAvis expects a gensim bag-of-words corpus (list of (term_id, count) tuples). If you see errors, confirm `corpus` is correctly built with `dictionary.doc2bow(...)`.
